### Testing the quality of the argument references.

In [2]:
import os
from nltk import tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
from sentence_transformers import SentenceTransformer,util
import re


##############################################################################################################


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")


##############################################################################################################


listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../structureWords.xlsx")

listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []

df_fileName = pd.DataFrame({})

for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TeachersMerged T1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TeachersMerged T1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

##############################################################################################################

corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

##############################################################################################################


k = 19

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

df_clusters.to_excel("TestClusterReference.xlsx")

##############################################################################################################


df_test = pd.DataFrame({})

df_ref = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\Teachers prompt\Goldstandards.xlsx")

def_clusters = pd.read_excel(r"TestClusterReference.xlsx")

listRef = df_ref['Gold standard']

listClusters = def_clusters['cluster']
listNewSentences = def_clusters['text']
listembReferences = []

for ref in listRef:
    listembReferences.append(model.encode(ref))

cosListRefToClusters = []
mm=0
countMissed =0
for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #this is the right reference cluster
    if (listMax[index_max] <0.5): #threshold to know if this item does not match any of the references!!!
        countMissed = countMissed +1
        df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": "NOT ASSIGNED"
              },ignore_index=True)
        continue

    cosListRefToClusters.append(index_max)
    df_test = df_test.append({
        "ID" : mm,
        "text": listNewSentences[mm],
        "cluster": listClusters[mm],
        "cos Sim": index_max
              },ignore_index=True)
    mm = mm+1


df_test.to_excel("TestClusterReference.xlsx");


OSError: Unable to load weights from pytorch checkpoint file for 'C:\Users\Ahmed/.cache\torch\sentence_transformers\sentence-transformers_all-mpnet-base-v2\pytorch_model.bin' at 'C:\Users\Ahmed/.cache\torch\sentence_transformers\sentence-transformers_all-mpnet-base-v2\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [10]:
print(countMissed)

13065
